<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [3]:
import pandas as pd

url = '/content/drive/MyDrive/규모별_미분양현황_20210514153511.csv'
col = ['~60m2', '60~85m2', '85m2~']
row = ['서울', '부산', '대구', '대전']
df = pd.read_csv(url,encoding='cp949')
df

,시도(1),시점,공공부문,공공부문.1,민간부문,민간부문.1,민간부문.2
0,시도(1),시점,60㎡이하,60∼85㎡,60㎡이하,60∼85㎡,85㎡초과
1,서울,2020. 08,0,0,54,2,0
2,부산,2020. 08,0,0,577,735,142
3,대구,2020. 08,0,0,143,1437,44
4,대전,2020. 08,0,0,782,1,0


In [4]:
df = pd.read_csv(url,encoding='cp949',skiprows=(1),index_col='시도(1)').iloc[:,3:] #skiprows의 사용법에 대해서 알아보자! usecols도 같이 알아보자!
#index_col parameter 사용하면, 해당 컬럼이 index로 간다.
df.columns = col
df

,~60m2,60~85m2,85m2~
시도(1),,,
서울,54,2,0
부산,577,735,142
대구,143,1437,44
대전,782,1,0


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [0+0+0+52+2+0, 0+0+0+590+665+142, 0+0+0+113+1061+42, 0+0+0+772+1+0]
```
을 비교 하게 될 것입니다.




In [5]:
from scipy.stats import chisquare 

# data_sum = [df.loc['서울'].sum(),df.loc['부산'].sum(),df.loc['대구'].sum(),df.loc['대전'].sum()] #data_sum = df.sum(axis=1)
data_sum = df.sum(axis=1)
print(data_sum)

chi1 = chisquare(data_sum,axis = None)
chi1

시도(1)
서울      56
부산    1454
대구    1624
대전     783
dtype: int64


Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

```
- 결론
  - 귀무가설: 지역별 미분양 주택수의 합은 같다. (고르게 분포)
  - 대립가설: 지역별 미분양 주택수의 합이 같지 않다. (고르게 분포 x)
  - 알파 : 0.05
  - pvalue = 0.0
  - 따라서 귀무가설 기각, 대립가설 채택
```

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [6]:
from scipy.stats import chi2_contingency

chi2 = chi2_contingency(df) #df -> 이미 crosstab이 되어있네.
chi2

(2064.576731417199, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
        [577.59101353, 807.36533061,  69.04365586],
        [645.12228746, 901.76155221,  77.11616033],
        [311.04110288, 434.77789124,  37.18100587]]))

```
- 결론
  - 귀무가설: 지역과 규모에 따른 미분양 수는 서로 독립적이다./관계가 없을 것이다.
  - 대립가설: 지역과 규모에 따른 미분양 수는 서로 관계가 있다./영향을 끼친다./독립적이지 않다./ 비독립적이다.
  - 알파: 0.05
  - pvalue: 0.0
  - 따라서 귀무가설 기각, 대립가설 채택
```

### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

이 후 `chi2_contingency()`의 결과물과 비교해보세요

$\chi^2 = \sum\frac{(obs-exp)^2}{exp}$ 

In [7]:
import numpy as np
import pandas as pd
from scipy import stats

## 넘파이를 통한 구현
chi = 0
obs = np.array(df)
exp = obs.sum()/4
for i in obs:
    chi += (np.power(i.sum()-exp,2)/exp)
print('Chi_Square:{0}, Pvalue:{1}'.format(chi,stats.chi2.pdf(chi,1)))

Chi_Square:1564.4572376818994, Pvalue:0.0


In [8]:
#또 다른 방법
obs = data_sum.to_numpy()#dataframe을 array의 형태로 바꿔주는 메소드 to_numpy()
exp = obs.mean()
squared = np.power(obs-exp,2)
print(squared)
chi = np.sum(squared/exp)

print('Chi_Square:{0}, Pvalue:{1}'.format(chi,stats.chi2.pdf(chi,1)))

[852390.5625 225387.5625 415702.5625  38514.0625]
Chi_Square:1564.4572376818994, Pvalue:0.0


## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [9]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]
import numpy as np

def myChisq(value):
    size_n = np.sum(value)
    mean_v = [size_n/len(value) for _ in range(len(value))]
    value = np.array(value)
    mean_v = np.array(mean_v)
    chi_s = np.sum(((value-mean_v)**2)/mean_v)
    return chi_s.sum()

 
print('Chi_Square_v1:{0}, Pvalue_v1:{1}'.format(myChisq(v1),stats.chi2.pdf(myChisq(v1),1)))
print('Chi_Square_v2:{0}, Pvalue_v2:{1}'.format(myChisq(v2),stats.chi2.pdf(myChisq(v2),1)))

Chi_Square_v1:2.3000000000000003, Pvalue_v1:0.0832928061175755
Chi_Square_v2:14.8, Pvalue_v2:6.33869772615072e-05


### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



## 3. Sign Test

아래 링크를 참조하여 Sign Test에 대한 글을 읽고

<https://en.wikipedia.org/wiki/Sign_test>

다음 데이터에서 두 그룹간에 차이가 있다에 대한 Sign Test의 과정과 결과를 설명하세요

|A|B|
|:-:|:-:|
|142|138|
|140|136|
|144|147|	
|144|139|	
|142|143|	
|146|141|	
|149|143|	
|150|145|	
|142|136|	
|148|146|	



In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
